# Using Keras Subclassing API
-----------------------------

In [1]:
import tensorflow as tf
from tensorflow import keras

Load MNIST dataset

In [2]:
mnist = tf.keras.datasets.mnist
(X_mnist_train, y_mnist_train), (X_mnist_test, y_mnist_test) = mnist.load_data()

Normalize the grayscale image

In [3]:
def normalize_grayscale(image_data):
    """
    Normalize the image data with Min-Max scaling to a range of [0.1, 0.9]
    :param image_data: The image data to be normalized
    :return: Normalized image data
    """
    a = 0.1
    b = 0.9
    image_data_min = image_data.min()
    image_data_max = image_data.max()
    return a + ((image_data - image_data_min)*(b-a))/(image_data_max - image_data_min)

train_mnist_features = normalize_grayscale(X_mnist_train)
test_mnist_features = normalize_grayscale(X_mnist_test)

### 1 - Create a Custom Layer

In [4]:
class MyCustomDense(tf.keras.layers.Layer):
    # Initialize this class with the number of units
    def __init__(self, units):
        super(MyCustomDense, self).__init__()
        self.units = units
 
    # Define the weights and the bias
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                            initializer='random_normal',
                            trainable=True)
        self.b = self.add_weight(shape=(self.units,),
                            initializer='random_normal',
                            trainable=True)
 
    # Applying this layer transformation to the input tensor
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
    # Function to retrieve the configuration
    def get_config(self):
        return {'units': self.units}

In [5]:
x = tf.ones((2,2))
my_custom_layer = MyCustomDense(4)
y = my_custom_layer(x)
print(y)

tf.Tensor(
[[-0.09472567  0.11487222 -0.14412683  0.03258352]
 [-0.09472567  0.11487222 -0.14412683  0.03258352]], shape=(2, 4), dtype=float32)


In [6]:
# Create an input layer
inputs = keras.Input((12,4))
# Add an instance of MyCustomeDense layer
outputs = MyCustomDense(2)(inputs)

# Create a model
model = keras.Model(inputs, outputs)

# Get the model config
config = model.get_config()

# Reload the model from the config
new_model = keras.Model.from_config(config, 
                                    custom_objects={'MyCustomDense': MyCustomDense})

### 2 - Create a Custom Model

In [4]:
class MyMNISTModel(tf.keras.Model):
  def __init__(self, num_classes):
    super(MyMNISTModel, self).__init__(name='my_mnist_model')
    self.num_classes = num_classes
    # Defining the layers
    self.flatten_1 = tf.keras.layers.Flatten()
    self.dropout = tf.keras.layers.Dropout(0.1)
    self.dense_1 = tf.keras.layers.Dense(50, activation='relu')
    self.dense_2 = tf.keras.layers.Dense(10, activation='softmax')

  # We define our forward pass using layers created in the init method
  def call(self, inputs, training=False):
    x = self.flatten_1(inputs)
    x = self.dense_1(x)
    # Apply dropout only during the training phase
    if training:
      x = self.dropout(x, training=training)
    return self.dense_2(x)
  

In [5]:
my_mnist_model = MyMNISTModel(10)

In [6]:
# Compile
my_mnist_model.compile(optimizer='sgd',
                       loss='sparse_categorical_crossentropy',
                       metrics=['accuracy'])

# Train
my_mnist_model.fit(train_mnist_features, y_mnist_train, 
                   validation_data=(test_mnist_features, y_mnist_test), 
                   epochs=10)

Epoch 1/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.8260 - accuracy: 0.7817 - val_loss: 0.4128 - val_accuracy: 0.8886
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4119 - accuracy: 0.8836 - val_loss: 0.3378 - val_accuracy: 0.9056
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3625 - accuracy: 0.8950 - val_loss: 0.3059 - val_accuracy: 0.9140
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3358 - accuracy: 0.9036 - val_loss: 0.2870 - val_accuracy: 0.9191
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3167 - accuracy: 0.9089 - val_loss: 0.2705 - val_accuracy: 0.9251
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3003 - accuracy: 0.9145 - val_loss: 0.2595 - val_accuracy: 0.9261
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2862 - accuracy: 0.9180 - val_loss: 0.2489 - val_accuracy: